## clustering attempt

In [1]:
import re
import json
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
from transformers import pipeline


In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
pd.set_option('display.max_colwidth', None)

Process:

1) want list of each article's title + description
2) clean data (lowercase, remove html, remove special chars)
3) preprocess (tokenize and remove stopwords, lemmatize)
4) use kmeans for clustering (q: fixed number of clusters?)
5) apply PCA
6) select only largest clusters?
7) web scrape articles belonging to that cluster
8) summarize with OpenAI

In [4]:
# Step 1: read in data
def read_data(file_path):
    with open(file_path) as f:
        data = json.load(f)
    return data

search_data = read_data('../newsapi/outputs/test_search_trump.json')

In [5]:
# Step 1 cont: convert to DF
articles_df = pd.json_normalize(search_data)


In [6]:
articles_df.head(5)

,author,title,description,url,urlToImage,publishedAt,content,user_pref,source.id,source.name
0,"Lauren Goode, Paresh Dave, Will Knight",What Donald Trump's Win Will Mean for Big Tech,Donald Trump's approach to Big Tech has oscillated between calls for stricter regulations for some players and a hands-off approach for others. Here's how he might steer tech policy in a second term.,https://www.wired.com/story/trump-tech-policy/,"https://media.wired.com/photos/672bcd1062c380013856bb0f/191:100/w_1280,c_limit/Business_bigtech_GettyImages-697900116.jpg",2024-11-07T20:00:17Z,"The most raucous cheers of the night were prompted by Trumps promise to fire Gary Gensler, chairman of the Securities and Exchange Commission, a regulatory agency that has brought a volley of lawsuit… [+2787 chars]",False,wired,Wired
1,Caroline Haskins,ICE Started Ramping Up Its Surveillance Arsenal Immediately After Donald Trump Won,US Immigration and Customs Enforcement put out a fresh call for contracts for surveillance technologies before an anticipated surge in the number of people it monitors ahead of deportation hearings.,https://www.wired.com/story/ice-surveillance-contracts-isap/,"https://media.wired.com/photos/6733a1802d34679fa5571df1/191:100/w_1280,c_limit/Security_ICE_GettyImages.jpg",2024-11-13T12:00:00Z,"Compared to its recent November notice, ICE was more detailed about its 2025 plans in a different notice released last year. This earlier notice shows that ICE was preparing for a more intense immigr… [+2023 chars]",False,wired,Wired
2,Kelly McEvers,The Great American Microchip Mobilization,"Under Donald Trump and Joe Biden alike, the US has been determined to “reshore” chipmaking. Now money and colossal infrastructure are flowing to a vast Intel site in Ohio—just as the company may be falling apart.",https://www.wired.com/story/intel-great-american-microchip-mobilization/,"https://media.wired.com/photos/67329e6ef0b163d7e81ffb79/191:100/w_1280,c_limit/Intel_Ohio_DSC09351.jpg",2024-11-14T11:00:00Z,"It took nearly two years of planning to get here. This is one of about two dozen superloadspieces of highway cargo that weigh more than 120,000 poundsthat will be lumbering across Ohio for Intel. Tod… [+2622 chars]",False,wired,Wired
3,Eric Geller,"More Spyware, Fewer Rules: What Trump’s Return Means for US Cybersecurity","Experts expect Donald Trump’s next administration to relax cybersecurity rules on businesses, abandon concerns around human rights, and take an aggressive stance against the cyber armies of US adversaries.",https://www.wired.com/story/trump-administration-cybersecurity-policy-reversals/,"https://media.wired.com/photos/6735e10f310e865b05b94244/191:100/w_1280,c_limit/Security_Animation_GettyImages.jpg",2024-11-14T10:30:00Z,"Trump is also unlikely to continue the Biden administrations campaign to limit the proliferation of commercial spyware technologies, which authoritarian governments have used to harass journalists, c… [+2985 chars]",False,wired,Wired
4,Leah Feiger,We Break Down the Internet’s Future Under Trump 2.0,"WIRED global editorial director Katie Drummond joins this week to discuss what the fragmented internet did for the Trump campaign, and what the incoming Trump administration means for the internet.",https://www.wired.com/story/the-internets-future-under-donald-trump/,"https://media.wired.com/photos/67328cfe8b3c4eb2afd88938/191:100/w_1280,c_limit/Politics-Lab-Umbrella-of-Election-Politics-2182522891.jpg",2024-11-14T18:58:19Z,"Leah Feiger: Enough trillions that don't actually currently exist in that budget.\r\nKatie Drummond: Exactly. So thinking about that very messy, very human business and Elon Musk and Donald Trump in a … [+5020 chars]",False,wired,Wired


In [7]:
# Step 2: clean and tokenize data
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\W+', ' ', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

articles_df['combined_text'] = articles_df['title'].fillna('') + ' ' + articles_df['description'].fillna('')
articles_df['processed_text'] = articles_df['combined_text'].apply(clean_text)

In [8]:
# Step 3: embed and cluster using DBSCAN (can also try kmeans)
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(articles_df['processed_text'].fillna(''))

dbscan = DBSCAN(eps=0.5, min_samples=3, metric='cosine')
clusters = dbscan.fit_predict(embeddings)

articles_df['cluster'] = clusters

In [16]:
articles_df['cluster'].value_counts()

cluster
-1    87
 0    13
 2     9
 5     6
 1     4
 4     4
 6     3
 3     3
Name: count, dtype: int64

In [17]:
print(articles_df[articles_df['cluster'] == 0]['title'])
print(articles_df[articles_df['cluster'] == 0]['description'])

0                                          What Donald Trump's Win Will Mean for Big Tech
3               More Spyware, Fewer Rules: What Trump’s Return Means for US Cybersecurity
46                           How will a second Trump presidency affect the tech industry?
51               Did you need another reminder that Donald Trump watches TV? Here you go.
59     Elon Musk went all-in on Pennsylvania — and his expensive effort paid off big time
68           Silicon Valley is betting a Musk-inspired Trump could unleash a startup boom
102                                       What Donald Trump’s Win Could Mean for Vaccines
103                             How long will Elon Musk and Donald Trump’s lovefest last?
105                                           What Donald Trump’s Win Means For Inflation
108                                    What Trump's win means for the consulting industry
109                                                  What Trump’s Win Means for Education
119       

In [18]:
print(articles_df[articles_df['cluster'] == 2]['title'])
print(articles_df[articles_df['cluster'] == 2]['description'])



22                                                             Watch: Putin congratulates 'courageous' Trump on election win
35                                                    Who is Pete Hegseth - the Fox News host who will be defence secretary?
42                                   'Crimea is gone': Trump adviser says Ukraine focus must be peace not retaking territory
64                                                              Inside Trump and Putin's first phone call since the election
67                                      With Trump's White House win, the clock is ticking on over $6 billion in Ukraine aid
70     Trump's Pentagon pick criticized US involvement in Ukraine, said Putin probably wouldn't go 'much further' if he wins
89                                                      Trump to nominate Fox News host Pete Hegseth to be defense secretary
115                                          Trump takes calls from growing list of world leaders following election victory


In [19]:
print(articles_df[articles_df['cluster'] == 5]['title'])
print(articles_df[articles_df['cluster'] == 5]['description'])

31                 Divided Arizona contends with Trump's sweeping border plan
79                                                  Donald Trump wins Arizona
110    Trump won a comprehensive victory. Can other Republicans replicate it?
117      More voters saw Trump as the candidate of change: Exit poll analysis
118                The Strategist Who Predicted Trump’s Multiracial Coalition
127                                  Why Democrats Are Losing the Culture War
Name: title, dtype: object
31                                                                                   The state is likely to be one places seeing a shift under the president-elect's immigration policies.
79     Arizona was that last state to be called by the Associated Press in the presidential race. It brings Trump's total electoral vote count to 312, with 226 for Vice President Harris.
110                                         Donald Trump scored a comprehensive victory Tuesday, improving his numbers across sever

In [20]:
print(articles_df[articles_df['cluster'] == 1]['title'])
print(articles_df[articles_df['cluster'] == 1]['description'])


8     Jeff Bezos says he’s a climate guy — why is he kissing the ring?
27                    Climate talks to open in shadow of Trump victory
29                     Trump victory 'major setback' to climate action
36    Climate fight 'bigger than one election', says Biden’s top envoy
Name: title, dtype: object
8     Jeff Bezos joined other tech moguls in congratulating Trump on his election victory. That doesn’t seem to jive with his “passion” for fighting climate change.
27                                                                   Leaders heading to the annual UN climate talks have a lot of other distractions to contend with
29                                   Trump's election will hit immediate efforts to tackle climate change, experts say - but the longer-term effect is less certain.
36                                                          COP29 climate summit opens amid fears the US election will derail efforts to stop the planet heating up.
Name: description, dtype: obj

In [21]:
print(articles_df[articles_df['cluster'] == 4]['title'])
print(articles_df[articles_df['cluster'] == 4]['description'])


63                                                 Matt Gaetz's most controversial moments
83                                     Trump announces Matt Gaetz as attorney general pick
87     Senate Republicans concerned with Gaetz nomination ask to access House ethics probe
126                                        Trump nominates Matt Gaetz for Attorney General
Name: title, dtype: object
63                                                                                 Matt Gaetz, Donald Trump's polarizing pick to be the next attorney general, has long been at the center of controversy.
83             President-elect Donald Trump announced his pick for one of the biggest jobs in his new administration: attorney general. And for that job, Trump has chosen Florida Congressman Matt Gaetz.
87     Senators are calling for access to a House Ethics Committee probe into former Rep. Matt Gaetz, R-Fla., following his nomination to be the next attorney general under President-elect Donald Trump.


In [24]:
# step 4: summarize main clusters (may use OpenAI for this in the future)
summaries = []
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
for cluster in articles_df['cluster'].unique():
    if cluster == -1:
        continue
    cluster_articles = articles_df[articles_df['cluster'] == cluster]['combined_text']
    combined_text = ' '.join(cluster_articles[:15])
    if combined_text.strip():
        summary = summarizer(combined_text, max_length=50, min_length=30, do_sample=False)
        summaries.append({'cluster': cluster, 'summary_text': summary[0]['summary_text']})
        print(f"Cluster {cluster} Summary: {summary[0]['summary_text']}")

Cluster 0 Summary:  Trump's approach to Big Tech has oscillated between calls for stricter regulations for some players and a hands-off approach for others . Silicon Valley is betting a Musk-inspired Trump could unleash a startup boom . The president-elect returned to
Cluster 1 Summary:  Jeff Bezos says he’s a climate guy — why is he kissing the ring? Jeff Bezos joined other tech moguls in congratulating Trump on his election victory . COP29 climate summit opens amid fears the US election will derail efforts
Cluster 2 Summary:  Pete Hegseth, a Fox News host, has advocated for a conservative cultural shift in the US military . On a podcast last week, he said he didn't want US intervention forcing Putin's hand . With Trump's White House
Cluster 5 Summary:  Arizona was that last state to be called by the Associated Press in the presidential race . It brings Trump's total electoral vote count to 312, with 226 for Vice President Harris .
Cluster 6 Summary:  Trump fans and Harris supporters 

In [ ]:
# PSA I tried it with OpenAI and summaries are much better. So we'll probably use that.

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

pd.set_option('display.max_colwidth', None)

model = SentenceTransformer('all-MiniLM-L6-v2')
dbscan = DBSCAN(eps=0.5, min_samples=3, metric='cosine')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [28]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\W+', ' ', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

dbscan2 = DBSCAN(eps=0.2, min_samples=5, metric='cosine')

def cluster_daily_news(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        articles_data = json.load(f)
    articles_df = pd.json_normalize(articles_data)
    articles_df['combined_text'] = articles_df['title'].fillna('') + ' ' + articles_df['content'].fillna('')
    articles_df['processed_text'] = articles_df['combined_text'].apply(clean_text)
    embeddings = model.encode(articles_df['processed_text'].fillna(''))
    clusters = dbscan.fit_predict(embeddings)
    articles_df['cluster'] = clusters
    id_to_cluster = dict(zip(articles_df.index, articles_df['cluster']))
    return id_to_cluster

def extract_first_sentence(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    sentences = nltk.sent_tokenize(text)
    return sentences[0] if sentences else ""

def cluster_daily_news_titles(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        articles_data = json.load(f)
    articles_df = pd.json_normalize(articles_data)
    articles_df['first_sentence'] = articles_df['content'].fillna('').apply(extract_first_sentence)
    articles_df['combined_text'] = articles_df['title'].fillna('') + " " + articles_df['first_sentence'].fillna('')
    articles_df['processed_text'] = articles_df['combined_text'].apply(clean_text)
    embeddings = model.encode(articles_df['processed_text'].fillna(''))
    clusters = dbscan2.fit_predict(embeddings)
    articles_df['cluster'] = clusters
    id_to_cluster = dict(zip(articles_df.index, articles_df['cluster']))
    return id_to_cluster

In [9]:
path = r"C:\Users\jared\OneDrive\Desktop\bitewise\python-api\data\articles_data.json"


In [7]:
cluster_dict = cluster_daily_news(path)
print(set(cluster_dict.values()))

{0, 1, 2, 3, -1}


In [36]:
cluster_dict_titles = cluster_daily_news_titles(path)
print(set(cluster_dict_titles.values()))

{0, 1, 2, 3, 4, -1}


In [34]:
def remove_articles_by_source(input_file, output_file, source_to_remove):
    # Read the JSON data
    with open(input_file, 'r', encoding='utf-8') as f:
        articles_data = json.load(f)
    
    # Filter out articles with the specified source
    filtered_articles = [
        article for article in articles_data if article.get('source') != source_to_remove
    ]
    
    # Save the cleaned data back to a new file (or overwrite the original file)
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(filtered_articles, f, indent=4, ensure_ascii=False)
    
    print(f"Removed articles from source: {source_to_remove}")
    print(f"Remaining articles: {len(filtered_articles)}")

path = r"C:\Users\jared\OneDrive\Desktop\bitewise\python-api\data\articles_data.json"
source_to_remove = 'https://www.businessinsider.com/'
remove_articles_by_source(path, path, source_to_remove)


Removed articles from source: https://www.businessinsider.com/
Remaining articles: 1264


In [37]:
with open(path, 'r',  encoding='utf-8') as f:
    articles_data = json.load(f)
articles_df = pd.DataFrame(articles_data)
articles_df['cluster'] = articles_df.index.map(cluster_dict_titles)

# get top clusters
top_clusters = (
    articles_df.loc[articles_df['cluster'] != -1, 'cluster']
    .value_counts()
    .nlargest(4)
    .index
)

# get articles for top clusters
top_articles = articles_df[articles_df['cluster'].isin(top_clusters)]

response = (
    top_articles.groupby('cluster')
    .apply(lambda x: x.to_dict(orient='records'))
    .to_dict()
)
response

C:\Users\jared\AppData\Local\Temp\ipykernel_7744\1716046395.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.to_dict(orient='records'))


{0: [{'url': 'https://abcnews.go.com/Politics/president-biden-pardons-family-members-final-minutes-presidency/story?id=117893348',
   'title': 'Biden pardons family members in final minutes of presidency',
   'source': 'https://abcnews.go.com/',
   'content': 'Biden issued pardons to several family members Monday. President Joe Biden issued pardons to several close family members Monday inthe final minutes of his presidency. "My family has been subjected to unrelenting attacks and threats, motivated solely by a desire to hurt me -- the worst kind of partisan politics," Biden wrote in a statement.\xa0"Unfortunately, I have no reason to believe these attacks will end." "That is why I am exercising my power under the Constitution to pardon James B. Biden, Sara Jones Biden, Valerie Biden Owens, John T. Owens, and Francis W. Biden," he continued. "The issuance of these pardons should not be mistaken as an acknowledgment that they engaged in any wrongdoing, nor should acceptance be misconstr

In [22]:
top_articles

url  \
3                                           https://mashable.com//article/notre-dame-ohio-state-college-football-2025-live-stream   
4                                                                  https://mashable.com//article/strands-nyt-hint-january-20-2025   
5                                                             https://mashable.com//article/melania-trump-hat-inauguration-outfit   
6                                                              https://fortune.com/2025/01/20/donald-trump-inauguration-speech-2/   
12                                                                 https://mashable.com//article/trump-executive-order-tiktok-ban   
...                                                                                                                           ...   
1310                                                      https://www.vogue.com//article/what-people-are-wearing-pre-inauguration   
1313      https://www.pbs.org/newshour/show/why-netanyahu-announced-a-last-minute-snag-in-israel-hamas-ceasefire-and-hostage-deal   
1316               https://www.esquire.com/news-politics//news-politics/politics/a60787873/republican-senators-visit-trump-trial/   
1318  https://www.salon.com//2025/01/20/biden-issues-clemency-to-family-democratic-politicians-and-leonard-peltier/?in_brief=true   
1320             https://www.esquire.com/news-politics//news-politics/politics/a63373441/trump-legal-battle-panama-hotel-project/   

                                                                                            title  \
3                                                   How to watch Notre Dame vs. Ohio State online   
4                                                       NYT Strands hints, answers for January 20   
5                           Melania Trump's inauguration hat draws comparisons to Carmen Sandiego   
6     Trump returns to power promising retribution, reversal of the Biden era, and a trip to Mars   
12                                                          Back like it never leftTikTok is back   
...                                                                                           ...   
1310                            The Most Notable Fashion Moments From the Pre-Inauguration Events   
1313        Why Netanyahu announced a last-minute snag in Israel-Hamas ceasefire and hostage deal   
1316                                                     Aww, Trump Brought Some Buddies to Court   
1318                                  Jan 20, 1:40 pm ESTBiden grants clemency to Leonard Peltier   
1320                                           Gee, I Wonder Why Trump Is So Interested in Panama   

                                      source  \
3                      https://mashable.com/   
4                      https://mashable.com/   
5                      https://mashable.com/   
6                       https://fortune.com/   
12                     https://mashable.com/   
...                                      ...   
1310                  https://www.vogue.com/   
1313           https://www.pbs.org/newshour/   
1316  https://www.esquire.com/news-politics/   
1318                  https://www.salon.com/   
1320  https://www.esquire.com/news-politics/   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   